<a href="https://colab.research.google.com/github/tyoc213/fastai_xla_extensions/blob/tracing/explore_nbs/tracing_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
VERSION = "20200516"  #@param ["1.5" , "20200516", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION > /dev/null

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  21858      0 --:--:-- --:--:-- --:--:-- 21765
Copying gs://tpu-pytorch/wheels/torch-nightly+20200516-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200516-cp36-cp36m-linux_x86_64.whl...
\ [1 files][119.8 MiB/119.8 MiB]                                                
Operation completed over 1 objects/119.8 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20200516-cp36-cp36m-linux_x86_64.whl...
/ [1 files][  2.3 MiB/  2.3 MiB]                                                
Operation completed over 1 objects/2.3 MiB.                                      
ERROR:

In [2]:
!pip install --upgrade https://github.com/tyoc213-contrib/fastai2/archive/more_begins.zip > /dev/null

In [3]:
!pip install --upgrade https://github.com/tyoc213/fastai_xla_extensions/archive/tracing.zip > /dev/null

In [4]:
import fastai_xla_extensions.core

In [5]:
from fastai2.vision.all import *

In [6]:
path = untar_data(URLs.MNIST)

In [7]:
datablock = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=GrandparentSplitter(train_name='training',valid_name='testing'),
    item_tfms=Resize(28),
    batch_tfms=[]
)

In [8]:
datablock_tfms= datablock.new(batch_tfms=[*aug_transforms(size=28, do_flip=False), Normalize],item_tfms=Resize(40))

In [9]:
dls1 = datablock_tfms.dataloaders(path,bs=32)


In [10]:
learn1 = cnn_learner(dls1, resnet18, metrics=accuracy)

In [11]:
l = []
ct = fastai_xla_extensions.core.ChromeTrace(l)
learn1.fine_tune(0, freeze_epochs=4, cbs=ct)


epoch,train_loss,valid_loss,accuracy,time
0,1.208226,0.761951,0.754200,10:11
1,0.701129,0.436309,0.860100,10:06
2,0.440163,0.219988,0.928100,10:09
3,0.307565,0.122406,0.962900,10:07


In [12]:
ct.export_chrome_trace()

# Now profile autograd (this never ends, RAM exhaustion)

In [ ]:
%%time
from fastai2.vision.all import *
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")
def label_func(f): return f[0].isupper()
dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224), batch_tfms=[*aug_transforms(size=28, do_flip=False), Normalize])
learn = cnn_learner(dls, resnet34, metrics=error_rate)

with torch.autograd.profiler.profile() as prof:
    learn.fine_tune(1)
    


epoch,train_loss,valid_loss,error_rate,time
0,1.040565,0.707499,0.314614,02:28


epoch,train_loss,valid_loss,error_rate,time


In [17]:
prof.export_chrome_trace("prof.json")

/content


In [18]:
!ls

export_chrome_trace.json
pytorch-xla-env-setup.py
sample_data
torch-nightly+20200516-cp36-cp36m-linux_x86_64.whl
torchvision-nightly+20200516-cp36-cp36m-linux_x86_64.whl
torch_xla-nightly+20200516-cp36-cp36m-linux_x86_64.whl


In [ ]:
print("done")